In [ ]:
# Bring in my Dependencies

import pandas as pd
import os, csv
from sqlalchemy import create_engine


In [ ]:
# Bring in my csv that I made in the 'CHG_Netflix_SAG_merged_data.' notebook, and create a dataframe
# from it. Additionally, we drop the Unnamed: 0 column that gets created in the process as it is
# unneeded.

data = os.path.join('Resources','cleaned_merged','cleaned_netflix_sag_data.csv')
data1 = pd.read_csv(data)
data_df = pd.DataFrame(data1).drop(columns='Unnamed: 0')
data_df

In [ ]:
# Below, I create a dataframe whose name and data match the SQL table that I have set up in pgadmin4

# The actors table needed to have the id and full_name variables. I dropped the duplicate values as I need each
# entry to be a unique identifier. I then set the index to 'id' which will be the primary key for the SQL table.
# Copy() is used to ensure I am not slicing the original data

actors = data_df[['id','full_name']].set_index('id').drop_duplicates().copy()

# For the titles dataframe, I followed the exact same method used to create my actors dataframe only using different
# variables 

titles = data_df[['show_id','title','type','release_year','description']].set_index('show_id').drop_duplicates().copy()

# And again for my act_ttl dataframe

act_ttl = data_df[['id','show_id']].set_index('id').drop_duplicates().copy()

# this table was a bit tricker, as I had to create a new unique id for the directors_names variable
# In order to accomplish this, I created the column director_id and set it to a number starting
# at 0 for each director name in the list. Once I set my index to director_id, I will have my primary key
#
# *NOTE* it is important to notice that I have not set the index for this table yet -
# that is because we need to reference this series later to build junction tables
# if I set the index now - I would have to reference it as an index, and I was struggling to 
# figure that one out. *NOTE*

directors = data_df[['director']].rename(columns={'director':'directors_names'}).drop_duplicates().copy()
directors['director_id'] = range(len(directors['directors_names']))


# Let's kick it up a notch. Now I need to create my dir_ttl table, and it will leverage data from 
# two of my created dataframes rather than the source data. To start, I took the show_id from the source
# data, drop_dups, copy - now I create a new series in the dir_ttl dataframe called director_id that is taken 
# from the directors table. I then drop any NULL values. Now, if I did not do the next line of code, my
# director_id series would all by float values, and I need them to be integers as they are primary keys.
# 
# To address this, I write a linear expression that converts every value of that series to an integer - then I set 
# the index to show_id, as it is the first primary key in the composite key
#
# *hint* this will be a junction table, i.e. both series are the primary keys of two other tables which make up this
# table's COMPOSITE KEY *hint*

dir_ttl = data_df[['show_id']].drop_duplicates().copy()
dir_ttl['director_id'] = directors['director_id']
dir_ttl = dir_ttl.dropna()
dir_ttl['director_id'] = [int(value) for value in directors['director_id']]
dir_ttl = dir_ttl.set_index('show_id')

# This process was relatively similar to the directors dataframe creation - I had to create a unique key by
# referencing a range of numbers that was equally as long as the dataframe so that it could be appended. Also
# like the directors table, I have held off on setting the index for this dataframe, as the series
# will need to be referenced later for junction table creation.

awards = data_df[['category','won']].copy()
awards['award_id'] = range(len(awards['won']))

# actors and awards junction table - this table carries the primary keys from the mentioned tables as its
# COMPOSITE key. 

act_award = data_df[['id']].copy()
act_award['award_id'] = awards['award_id']
act_award = act_award.set_index('id')

# directors and awards junction table - this table carries the primary keys from the mentioned tables as its
# COMPOSITE key. 

dir_award = directors[['director_id']].copy()
dir_award['award_id'] = awards['award_id']
dir_award = dir_award.set_index('director_id')

# With all of my junction tables created, I can go ahead and set the index for the awards and 
# directors dataframes.

awards = awards.set_index('award_id')
directors = directors.set_index('director_id')

In [ ]:
# my data had a bunch of strings that were used for the various award categories
# so I wrote this code in order to have a cleaner result in my queries by combining
# results that were similiar in nature 

awards['category'] = awards['category'].replace(
    {'MALE SUPPORT':'MALE SUPPORTING ROLE',
    ' MALE ACTOR IN A SUPPORTING ROLE':'MALE SUPPORTING ROLE',
    'FEMALE SUPPORT':'FEMALE SUPPORTING ROLE',
    ' FEMALE ACTOR IN A SUPPORTING ROLE':'FEMALE SUPPORTING ROLE',
    'FEMALE LEAD':'FEMALE LEAD ROLE',
    'FEMALE ACTOR IN A LEADING ROLE':'FEMALE LEAD ROLE',
    ' FEMALE ACTOR IN A LEADING ROLE':'FEMALE LEAD ROLE',
    'FEMALE LEAD IN A MOTION PICTURE':'FEMALE LEAD ROLE',
    'MALE LEAD':'MALE LEAD ROLE',
    ' MALE ACTOR IN A LEADING ROLE':'MALE LEAD ROLE',
    'MALE ACTOR IN A LEADING ROLE':'MALE LEAD ROLE'})

In [ ]:
# establish connection string, and create engine

conn = "postgres:postgres@localhost:5432/netflix_sag_db"
engine = create_engine(f'postgresql://{conn}')

In [ ]:
# inspect my database's tables to verify my connection is functioning

engine.table_names()

In [ ]:
# with my connection established - I can now insert my dataframes into my various sql database tables
# it is important to run the lines in the below order so that all of the tables connect properly

actors.to_sql(name='actors', con=engine, if_exists='append', index=True)
titles.to_sql(name='titles', con=engine, if_exists='append', index=True)
act_ttl.to_sql(name='act_ttl', con=engine, if_exists='append', index=True)
directors.to_sql(name='directors', con=engine, if_exists='append', index=True)
dir_ttl.to_sql(name='dir_ttl', con=engine, if_exists='append', index=True)
awards.to_sql(name='awards', con=engine, if_exists='append', index=True)
act_award.to_sql(name='act_award', con=engine, if_exists='append', index=True)
dir_award.to_sql(name='dir_award', con=engine, if_exists='append', index=True)